In [1]:
from main import get_miner
from log_parsing import from_id_to_template
import joblib
from data_integrate import *
from pattern_miner import *
from pattern_ranker2 import *
import pandas as pd
import json

In [2]:
# Reading from a file
with open("input/root_cause_hipster.json", 'r') as file:
    rc_dict = json.load(file)

In [3]:
log_template_miner = get_miner()

In [4]:
res_dict = joblib.load('output/fault_free_eventgraphs.joblib')

In [5]:
res_dict2 = joblib.load('output/fault_suffering_eventgraphs.joblib')

In [6]:
res_dict2.keys()

dict_keys(['2022-08-22 03:54', '2022-08-22 03:55', '2022-08-22 04:03', '2022-08-22 04:04', '2022-08-22 04:11', '2022-08-22 04:12', '2022-08-22 04:19', '2022-08-22 04:20', '2022-08-22 04:28', '2022-08-22 04:29', '2022-08-22 04:37', '2022-08-22 04:38', '2022-08-22 04:45', '2022-08-22 04:46', '2022-08-22 04:53', '2022-08-22 04:54', '2022-08-22 05:02', '2022-08-22 05:03', '2022-08-22 05:10', '2022-08-22 05:11', '2022-08-22 05:19', '2022-08-22 05:20', '2022-08-22 05:28', '2022-08-22 05:29', '2022-08-22 05:36', '2022-08-22 05:37', '2022-08-22 05:44', '2022-08-22 05:45', '2022-08-22 05:53', '2022-08-22 05:54', '2022-08-22 06:02', '2022-08-22 06:03', '2022-08-22 06:36', '2022-08-22 06:37', '2022-08-22 06:53', '2022-08-22 06:54', '2022-08-22 07:02', '2022-08-22 07:03', '2022-08-22 07:11', '2022-08-22 07:12', '2022-08-22 07:18', '2022-08-22 07:19', '2022-08-22 07:27', '2022-08-22 07:28', '2022-08-22 07:45', '2022-08-22 07:46', '2022-08-22 07:54', '2022-08-22 07:55', '2022-08-23 12:02', '2022-08-

In [7]:
abnormal_dates = res_dict2.keys()

In [8]:
min_score= 0.5

In [9]:
abnormal_dates = res_dict2.keys()
features_labeled = []
for normal_date in res_dict.keys():
    date = normal_date.split(' ')[0]
    fault_free_graphs = res_dict[normal_date]
    fault_free_pattern = get_pattern_support(fault_free_graphs)
    print(date)
    for abnormal_date in [d for d in abnormal_dates if d.split(' ')[0] == date]:
        print(abnormal_date)
        inject_pod = res_dict2[abnormal_date]['inject_pod']
        inject_svc = inject_pod.split("-")[0]
        inject_type = res_dict2[abnormal_date]['inject_type']
        root_cause =  rc_dict[inject_svc][inject_type].split('_')
        alarm_data_path = "input/fault_suffering"
        alarm_metric_list = get_metric_with_time(abnormal_date, alarm_data_path)
        alarm_list = generate_alarm(alarm_metric_list)
        fault_suffering_event_graphs = res_dict2[abnormal_date]['event_graphs']
        fault_suffering_pattern = get_pattern_support(fault_suffering_event_graphs)
        ranking_moreoften = pattern_ranker_custom(fault_free_pattern, fault_free_graphs, fault_suffering_pattern, log_template_miner, alarm_list, min_score=min_score)
        ranking_lessoften = pattern_ranker_custom(fault_suffering_pattern, fault_suffering_event_graphs, fault_free_pattern, log_template_miner, alarm_list, min_score=min_score)
        res = { 
               'normal_time': normal_date,
               'abnormal_time': abnormal_date,
               'result_list_inhealthy_moreoften': ranking_moreoften,
               'result_list_inhealthy_lessoften': ranking_lessoften,
               'inject_time': res_dict2[abnormal_date]['inject_time'],
               'inject_pod': inject_pod, 
               'inject_type': inject_type,
               'root_cause': root_cause
            }
        features_labeled.append(res)

2022-08-22
2022-08-22 03:54


/home/paul/miniconda3/envs/ml_pyth36/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


2022-08-22 03:55
2022-08-22 04:03
2022-08-22 04:04


[ERROR]2024-11-23 15:49:47,969 pattern_ranker2.py:188: Catch an exception: pop index out of range


2022-08-22 04:11
2022-08-22 04:12
2022-08-22 04:19
2022-08-22 04:20
2022-08-22 04:28


...

2022-08-22 04:45
2022-08-22 04:46
2022-08-22 04:53
2022-08-22 04:54
2022-08-22 05:02
2022-08-22 05:03
2022-08-22 05:10
2022-08-22 05:11
2022-08-22 05:19
2022-08-22 05:20
2022-08-22 05:28
2022-08-22 05:29
2022-08-22 05:36
2022-08-22 05:37
2022-08-22 05:44
2022-08-22 05:45
2022-08-22 05:53
2022-08-22 05:54
2022-08-22 06:02
2022-08-22 06:03
2022-08-22 06:36
2022-08-22 06:37
2022-08-22 06:53
2022-08-22 06:54
2022-08-22 07:02
2022-08-22 07:03
2022-08-22 07:11
2022-08-22 07:12
2022-08-22 07:18
2022-08-22 07:19
2022-08-22 07:27
2022-08-22 07:28
2022-08-22 07:45
2022-08-22 07:46
2022-08-22 07:54


[ERROR]2024-11-23 15:58:05,121 pattern_ranker2.py:188: Catch an exception: pop index out of range


2022-08-22 07:55
2022-08-23
2022-08-23 12:02
2022-08-23 12:03
2022-08-23 12:11
2022-08-23 12:12
2022-08-23 12:19
2022-08-23 12:20
2022-08-23 12:28
2022-08-23 12:29
2022-08-23 12:36
2022-08-23 12:37
2022-08-23 12:46
2022-08-23 12:47
2022-08-23 12:54
2022-08-23 12:55
2022-08-23 13:03
2022-08-23 13:04
2022-08-23 13:11
2022-08-23 13:12
2022-08-23 13:20
2022-08-23 13:21
2022-08-23 13:28
2022-08-23 13:29
2022-08-23 13:38
2022-08-23 13:39
2022-08-23 13:46
2022-08-23 13:47
2022-08-23 13:55
2022-08-23 13:56
2022-08-23 14:04
2022-08-23 14:05
2022-08-23 14:13
2022-08-23 14:14
2022-08-23 14:22
2022-08-23 14:23
2022-08-23 14:31
2022-08-23 14:32
2022-08-23 14:48
2022-08-23 14:49
2022-08-23 14:56
2022-08-23 14:57
2022-08-23 15:05
2022-08-23 15:06
2022-08-23 15:13
2022-08-23 15:14
2022-08-23 15:22
2022-08-23 15:23
2022-08-23 15:30
2022-08-23 15:31
2022-08-23 15:39
2022-08-23 15:40
2022-08-23 15:48
2022-08-23 15:49
2022-08-23 15:57
2022-08-23 15:58
2022-08-23 16:06
2022-08-23 16:07
2022-08-23 16:14
202

[ERROR]2024-11-23 16:09:58,710 pattern_ranker2.py:188: Catch an exception: pop index out of range


2022-08-23 16:42


In [10]:
# Convert to DataFrame
df = pd.DataFrame(features_labeled)
# locate templates with err or fail in string, treat them special (dont lose on ranking)

In [11]:
mask = df.result_list_inhealthy_moreoften.apply(lambda x: any(['resource_alert' in y and "Net" in y['resource_alert'] for y in x]))
pd.set_option('display.max_colwidth', 1000)
df[mask].head()

,abnormal_time,inject_pod,inject_time,inject_type,normal_time,result_list_inhealthy_lessoften,result_list_inhealthy_moreoften,root_cause
8,2022-08-22 04:28,cartservice-579f59597d-wc2lz,2022-08-22 04:27:03,network_delay,2022-08-22 03:51,"[{'events': '102_199', 'score': 1.0, 'deepth': 6, 'pod': 'cartservice-579f59597d-wc2lz', 'resource_alert': 'NetworkP90(ms)'}, {'events': '183_199', 'score': 1.0, 'deepth': 5, 'pod': 'cartservice-579f59597d-wc2lz', 'resource_alert': 'NetworkP90(ms)'}, {'events': '23_100', 'score': 0.5806451612903226, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '23_20'}, {'events': '4_40', 'score': 0.5538461538461539, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '38_84', 'score': 0.5412445730824892, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '23_106', 'score': 0.5203605843953995, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd', 'events_actual': '23_20'}, {'events': '38_78', 'score': 0.5190039318479686, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '4_38', 'score': 0.5069124423963134, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}]","[{'events': '102_25', 'score': 1.0, 'deepth': 6, 'pod': 'cartservice-579f59597d-wc2lz', 'resource_alert': 'NetworkP90(ms)', 'events_actual': '102_199'}, {'events': '104_106', 'score': 0.5822637106184364, 'deepth': 2, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '24_100', 'score': 0.5454545454545454, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '4_50', 'score': 0.5454545454545454, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_40'}, {'events': '4_3', 'score': 0.5272727272727272, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_40'}, {'events': '4_42', 'score': 0.5252525252525253, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_40'}, {'events': '16_41', 'score': 0.5252525252525253, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm'}, {'events': '4_18', 'score': 0.5186813186813187, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_40'}, {'events': '120_198', 'score': 0.5183333...",[NetworkP90(ms)]
9,2022-08-22 04:29,cartservice-579f59597d-wc2lz,2022-08-22 04:27:03,network_delay,2022-08-22 03:51,"[{'events': '102_199', 'score': 1.0, 'deepth': 6, 'pod': 'cartservice-579f59597d-wc2lz', 'resource_alert': 'NetworkP90(ms)'}, {'events': '183_199', 'score': 1.0, 'deepth': 5, 'pod': 'cartservice-579f59597d-wc2lz', 'resource_alert': 'NetworkP90(ms)'}, {'events': '169_199', 'score': 1.0, 'deepth': 2, 'pod': 'cartservice-579f59597d-wc2lz', 'resource_alert': 'NetworkP90(ms)'}, {'events': '4_40', 'score': 0.5671641791044776, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '28_208', 'score': 0.5185185185185185, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)', 'events_actual': '28_120'}, {'events': '4_42', 'score': 0.5185185185185185, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_50'}, {'events': '16_41', 'score': 0.5185185185185185, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '16_15'}, {'events': '23_106', 'score': 0.50765794511806, 'deepth': 3, 'pod': 'currencyserv...","[{'events': '102_25', 'score': 1.0, 'deepth': 6, 'pod': 'cartservice-579f59597d-wc2lz', 'resource_alert': 'NetworkP90(ms)', 'events_actual': '102_199'}, {'events': '24_100', 'score': 1.0, 'deepth': 3, 'pod': 'currencyservice-cf787dd48-vpjrd'}, {'events': '4_3', 'score': 0.5576923076923077, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}, {'events': '4_38', 'score': 0.5404040404040404, 'deepth': 1, 'pod': 'frontend-579b9bff58-t2dbm', 'events_actual': '4_42'}, {'events': '120_198', 'score': 0.5183333333333333, 'deepth': 4, 'pod': 'adservice-5f6585d649-fnmft', 'resource_alert': 'MemoryUsageRate(%)'}, {'events': '104_106', 'score': 0.5165631469979296, 

In [12]:
df.to_csv(f'output/features_labeled.csv', index=False)